In [23]:
%pylab inline
import numpy as np

num_egs = 5
num_features = 1
num_classifiers = 3

X = np.zeros((num_egs,num_features))
labels = np.zeros(num_egs)
X[0][0]=10; labels[0] = 1;
X[1][0]=30; labels[1] = 1;
X[2][0]=40; labels[2] = -1;
X[3][0]=60; labels[3] = -1;
X[4][0]=90; labels[4] = 1;

# X[0][1]=40; 
# X[1][1]=50; 
# X[2][1]=40; 
# X[3][1]=50; 
# X[4][1]=40; 

# X[0][0]=10; labels[0] = 1;
# X[1][0]=20; labels[1] = -1;
# X[2][0]=30; labels[2] = 1;


Populating the interactive namespace from numpy and matplotlib


In [76]:
# C++ like data structure
class Decision_Function():
    def __init__(self, error, direction, split_val, feature_index = -99, alpha=1):
        self.error = error
        self.direction = direction
        self.split_val = split_val
        self.feature_index = feature_index
        self.alpha = alpha


class DecisionStump():
    def __init__(self, criteria='gini'):
        self.criteria = criteria

    def get_ginni_curr_feature_val(self, labels, weights, feature_vals, split_value):
        left = []
        right = []
        leftweight = []
        rightweight = []
        gini = 0
        for i in range(self.num_egs):
            if (feature_vals[i] <= split_value):
                left.append(labels[i])
                leftweight.append(weights[i])
            else:
                right.append(labels[i])
                rightweight.append(weights[i])
        for group, weight in ([(left, leftweight), (right, rightweight)]):
            size = float(len(group))
            wt_of_group = sum([i for i in weight])
            if size == 0:
                continue
            score = 0.0
            p1 = 0
            p2 = 0
            for i, label in enumerate(group):
                if label == 1:
                    p1 += weight[i] * 1/wt_of_group
                else:
                    p2 += weight[i] * 1/wt_of_group
            score = p1 * p1 + p2 * p2
#             score = p1 * p1
    
            gini += (1.0 - score) * wt_of_group
            return gini
        
    def get_error_curr_feature_val(self, labels, weights, feature_vals, split_value):
        curr_error = 0
        for i in range(self.num_egs):
            # x[i] < c should be -1
            if  ((feature_vals[i] <= split_value) and (labels[i]!=-1)) or \
                ((feature_vals[i] > split_value) and (labels[i]!=1)):
                curr_error += weights[i];
            
        sol = Decision_Function(curr_error, 1, split_value)
        print("Error : {:.2f} \t Gini : {:.2f}".format(curr_error, self.get_ginni_curr_feature_val(labels, weights, feature_vals, split_value)))
        if curr_error > 0.5:
            sol.direction = -1
            sol.error = 1 - curr_error
        return sol
        
    
    def get_error_curr_feature(self, labels, weights, feature_vals, feature_split_values):
        best_decision_function_curr_feature = Decision_Function(float('inf'), 1, float('inf'))
        
        for i in range(len(feature_split_values)):
            curr_decision_function = self.get_error_curr_feature_val(labels, weights, feature_vals, feature_split_values[i])
            
            if curr_decision_function.error < best_decision_function_curr_feature.error:
                best_decision_function_curr_feature = curr_decision_function
            
        return best_decision_function_curr_feature
    
    def fit(self, labels, X_T, weights, feature_split_values):
        
        self.num_features = len(X_T)
        self.num_egs = len(X_T[0])
        
        best_decision_function = Decision_Function(float('inf'), 1, float('inf'))
        
        for i in range(self.num_features):
            current_feature_stump = self.get_error_curr_feature(labels, weights, X_T[i], 
                                                            feature_split_values[i])
            if current_feature_stump.error < best_decision_function.error:
                best_decision_function = current_feature_stump
                best_decision_function.feature_index = i
        return best_decision_function

class AdaBoost():
    def __init__(self, num_classifiers=3):
        self.num_classifiers = num_classifiers
    
    def get_feature_split_vals(self, X_T):
        '''
            Args:
                x : dataset
                
            Returns:
                feature_midpoints : For each feature returns a sorted list of mid points 
                                    of every two consecutive unique features
            
            Complexity:
               (Old) :  M + N * M + M * NlogN + M * N
               (New) : M * N log N + M * N
        '''
        
        X_copy = []     
        feature_midpoints = []

        # M * n log * n
        for i in range(self.num_features):
            X_copy.append(sorted(X_T[i]))
        
        # Get mid points of sorted unique values M * N
        for i in range(self.num_features):
            current = []
            j = 0
            while (j < self.num_egs - 1):
                k = j + 1
                while(X_copy[i][j] == X_copy[i][k] and k < self.num_egs - 1):
                    k += 1
                midpoint = (X_copy[i][j] + X_copy[i][k])/2
                current.append(midpoint)
                j = k
            feature_midpoints.append(current)
                            
        return feature_midpoints
    
    def update_weights(self,classifier, X_T, weights, labels):

        Z = 0
        feature_index = classifier.feature_index
        
        for i in range(self.num_egs):
            if classifier.direction == 1:
                if X_T[feature_index][i] <= classifier.split_val:
                    weights[i] *= np.exp(classifier.alpha * labels[i])
                else:
                    weights[i] *= np.exp(-classifier.alpha * labels[i])
            elif classifier.direction == -1:
                if X_T[feature_index][i] >= classifier.split_val:
                    weights[i] *= np.exp(classifier.alpha * labels[i])
                else:
                    weights[i] *= np.exp(-classifier.alpha * labels[i])
            Z += weights[i]

        return weights/Z

    
    def fit_weak_classifers(self, labels, X_T, feature_split_values):
        weights = np.ones(self.num_egs)/(self.num_egs)
        classifiers = []
        for t in range(self.num_classifiers): 
            clf = DecisionStump()
            classifiers.append(clf.fit(labels, X_T, weights, feature_split_values))
            alpha = np.log((1 - classifiers[-1].error)/classifiers[-1].error) * 0.5
            classifiers[-1].alpha = alpha
            weights = self.update_weights(classifiers[-1], X_T, weights, labels) 

#             print("t : {} \t alpha : {:.2f} \t ft_index : {} \t direction : {:2} \t split_value : {} \t error : {:.2f}".format(
#             t, classifiers[-1].alpha, classifiers[-1].feature_index, classifiers[-1].direction, classifiers[-1].split_val, classifiers[-1].error))
            
        self.classifiers = classifiers
    
    def fit(self, X, labels):
        self.num_egs = len(X)
        self.num_features = len(X[0])
        X_T = X.T
        feature_split_values = self.get_feature_split_vals(X_T)
        classifiers = self.fit_weak_classifers(labels, X_T,feature_split_values )
            
#         for t in range(self.num_classifiers):
            

In [75]:
clf = AdaBoost(num_classifiers=10)
clf.fit(X,labels)

Error : 0.60 	 Gini : 0.00
Error : 0.80 	 Gini : 0.00
Error : 0.60 	 Gini : 0.33
Error : 0.40 	 Gini : 0.60
Error : 0.37 	 Gini : 0.00
Error : 0.50 	 Gini : 0.00
Error : 0.37 	 Gini : 0.21
Error : 0.25 	 Gini : 0.37
Error : 0.42 	 Gini : 0.00
Error : 0.67 	 Gini : 0.00
Error : 0.58 	 Gini : 0.15
Error : 0.50 	 Gini : 0.29
Error : 0.31 	 Gini : 0.00
Error : 0.50 	 Gini : 0.00
Error : 0.44 	 Gini : 0.12
Error : 0.37 	 Gini : 0.22
Error : 0.50 	 Gini : 0.00
Error : 0.64 	 Gini : 0.00
Error : 0.54 	 Gini : 0.18
Error : 0.44 	 Gini : 0.34
Error : 0.39 	 Gini : 0.00
Error : 0.50 	 Gini : 0.00
Error : 0.42 	 Gini : 0.14
Error : 0.34 	 Gini : 0.26
Error : 0.46 	 Gini : 0.00
Error : 0.62 	 Gini : 0.00
Error : 0.56 	 Gini : 0.11
Error : 0.50 	 Gini : 0.22
Error : 0.37 	 Gini : 0.00
Error : 0.50 	 Gini : 0.00
Error : 0.45 	 Gini : 0.09
Error : 0.40 	 Gini : 0.17
Error : 0.50 	 Gini : 0.00
Error : 0.60 	 Gini : 0.00
Error : 0.54 	 Gini : 0.12
Error : 0.47 	 Gini : 0.23
Error : 0.42 	 Gini : 0.00
E

In [77]:
clf = AdaBoost(num_classifiers=10)
clf.fit(X,labels)

Error : 0.60 	 Gini : 0.00
Error : 0.80 	 Gini : 0.00
Error : 0.60 	 Gini : 0.27
Error : 0.40 	 Gini : 0.40
Error : 0.37 	 Gini : 0.00
Error : 0.50 	 Gini : 0.00
Error : 0.37 	 Gini : 0.17
Error : 0.25 	 Gini : 0.25
Error : 0.42 	 Gini : 0.00
Error : 0.67 	 Gini : 0.00
Error : 0.58 	 Gini : 0.14
Error : 0.50 	 Gini : 0.25
Error : 0.31 	 Gini : 0.00
Error : 0.50 	 Gini : 0.00
Error : 0.44 	 Gini : 0.11
Error : 0.37 	 Gini : 0.19
Error : 0.50 	 Gini : 0.00
Error : 0.64 	 Gini : 0.00
Error : 0.54 	 Gini : 0.16
Error : 0.44 	 Gini : 0.27
Error : 0.39 	 Gini : 0.00
Error : 0.50 	 Gini : 0.00
Error : 0.42 	 Gini : 0.13
Error : 0.34 	 Gini : 0.22
Error : 0.46 	 Gini : 0.00
Error : 0.62 	 Gini : 0.00
Error : 0.56 	 Gini : 0.11
Error : 0.50 	 Gini : 0.19
Error : 0.37 	 Gini : 0.00
Error : 0.50 	 Gini : 0.00
Error : 0.45 	 Gini : 0.09
Error : 0.40 	 Gini : 0.16
Error : 0.50 	 Gini : 0.00
Error : 0.60 	 Gini : 0.00
Error : 0.54 	 Gini : 0.11
Error : 0.47 	 Gini : 0.20
Error : 0.42 	 Gini : 0.00
E